In [1]:
from weight_avg_trees import LinearModel, weight_avg_2, weight_avg_4, weight_avg_8, regular
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

device_in_use = 'cuda'

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

# Load the data
ch = fetch_california_housing()
df = pd.DataFrame(data=ch.data, columns=ch.feature_names)
# Assuming `ch.target` is the target variable
df['Target'] = ch.target

# Splitting the dataset
df_train, df_temp = train_test_split(df, train_size=0.70, random_state=42)
df_val, df_test = train_test_split(df_temp, train_size=0.5, random_state=42)

# Separate the target variable
y_train = df_train['Target'].values
y_val = df_val['Target'].values
y_test = df_test['Target'].values

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the training data (excluding the target variable) and transform it
scaled_train_features = scaler.fit_transform(df_train.drop(columns=['Target']))

# Transform the validation and test data (excluding the target variable)
scaled_val_features = scaler.transform(df_val.drop(columns=['Target']))
scaled_test_features = scaler.transform(df_test.drop(columns=['Target']))

# Recombine scaled features with target variable
df_scaled_train = pd.DataFrame(scaled_train_features, columns=df_train.columns[:-1])  # Excluding the target variable column
df_scaled_train['Target'] = y_train

df_scaled_val = pd.DataFrame(scaled_val_features, columns=df_val.columns[:-1])
df_scaled_val['Target'] = y_val

df_scaled_test = pd.DataFrame(scaled_test_features, columns=df_test.columns[:-1])
df_scaled_test['Target'] = y_test

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.features = dataframe.drop('Target', axis=1).values
        self.labels = dataframe['Target'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float), torch.tensor(self.labels[idx], dtype=torch.float)

train_dataset = CustomDataset(df_scaled_train)
val_dataset = CustomDataset(df_scaled_val)
test_dataset = CustomDataset(df_scaled_test)

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [4]:
# Define a function to move an entire dataset to the device in advance
def preload_dataset_to_device(loader, batch_size, device):
    preloaded_data = [(inputs.to(device), targets.to(device)) for inputs, targets in loader]
    return DataLoader(preloaded_data, batch_size=batch_size)

# Preload datasets to device (if they fit into your device memory)
train_loader = preload_dataset_to_device(train_loader, len(train_dataset), device_in_use)
val_loader = preload_dataset_to_device(val_loader, len(val_dataset), device_in_use)

In [13]:
scaled_train_features.shape[1]

8

In [4]:
input_size = scaled_train_features.shape[1]
output_size = 1  

train_loss_traditional = []
test_loss_traditional = []

train_loss_2 = []
test_loss_2 = []

train_loss_4 = []
test_loss_4 = []

train_loss_8 = []
test_loss_8 = []

#105 is the lcm(3,7,15)

from tqdm import tqdm
for i in tqdm(range(20000)):

    testloss, trainloss = regular(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_traditional.append(trainloss) 
    test_loss_traditional.append(testloss)

    testloss, trainloss = weight_avg_2(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_2.append(trainloss) 
    test_loss_2.append(testloss)

    testloss, trainloss = weight_avg_4(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_4.append(trainloss) 
    test_loss_4.append(testloss)

    testloss, trainloss = weight_avg_8(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_8.append(trainloss) 
    test_loss_8.append(testloss)


print("Traditional:",np.mean(test_loss_traditional),np.std(test_loss_traditional))
print("Weight Avg 2 Base Models:",np.mean(test_loss_2),np.std(train_loss_2))
print("Weight Avg 4 Base Models:",np.mean(test_loss_4),np.std(train_loss_4))
print("Weight Avg 8 Base Models:",np.mean(test_loss_8),np.std(train_loss_8))

 38%|███▊      | 7658/20000 [3:30:08<5:38:40,  1.65s/it]


KeyboardInterrupt: 

In [9]:
ch.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [5]:
print("Traditional:",np.mean(test_loss_traditional),np.std(test_loss_traditional))
print("Weight Avg 2 Base Models:",np.mean(test_loss_2),np.std(train_loss_2))
print("Weight Avg 4 Base Models:",np.mean(test_loss_4),np.std(train_loss_4))

Traditional: 0.9670778355887366 0.052957925827381534
Weight Avg 2 Base Models: 0.9496065442012289 0.02592568634881894
Weight Avg 4 Base Models: 0.9437944365251342 0.012429520639051917


In [6]:
len(test_loss_traditional)

7658